#### 1. Locking Read 示例  
* 场景一 (for share):  
  	* 假设想要在一个名叫 child 的表中插入新 row, 前提要确保在名叫 parent 的表中包含一个 parent row. 想要确保引用的完整性, 可以按照如下步骤组织代码: 
	* 首先, 不能使用 noblocking read 读取 parent row 的方式, 检查 parent row 是否存在. 因为其他的事务可能在你 select 和 insert 操作之间, 把 parent row 删除了. 而你这个事务却意识不到; 为了避免这个问题, 要使用 `select ... for share` 锁定读读取 parent 表中 name = 'jones' 的行    
	```sql
	SELECT * FROM parent WHERE NAME = 'Jones' FOR SHARE;
	```
	在 `for share` 查询返回 'jones' 后, 就可以安全的在 child 表中 insert 一个 child 行, 最后提交事务. 任何想要在 parent 表的 'jones' 行上获取 X 锁的事务, 都要等待你这个事务完成. 换句话说: 其他事务要等到所有表到达一致性后侧能继续进行.

* 场景二 (for update): 
	* 假设有一个计数表 `CHILD_CODES`, 每次从这个表获取 id 的最新值作为主键插入到 child 表中, 然后让 `CHILD_CODES` 表的计数值 +1;     
	* 这种情况, `select ... for share` 就不适用了. 因为两个事务可能在同一时间读到相同的值, 从而导致 child 表的主键重复. 所以应该使用 `for update` 形式的锁定读, 最后在事务中 update 即可  
	```sql
	SELECT counter_field FROM child_codes FOR UPDATE;
	UPDATE child_codes SET counter_field = counter_field + 1;
	```




#### 2. 让 Locking Read 并发 -   NOWAIT 和 SKIP LOCKED   
因为使用 `SELECT ... FOR UPDATE` 或 `SELECT ... FOR SHARE` 锁定读时, 当被读取的 row 被其它事务加锁时, 就必须等待获得所的事务获取锁才能返回, 这种等待有时并不必要. 我们想让锁定读立刻返回, 或者仅仅是想看看查询的结果是否可用. 为了避免等待锁释放, 可以在锁定读上加上 `NOWAIT` 和 `SKIP LOCKED` 选项.
* NOWAIT   
  使用 `NOWAIT` 的锁定读不会等待获取锁, 会立刻返回. 如果被读的行被锁定, 则返回异常  
* SKIP LOCKED  
  使用 `NOWAIT` 的锁定读不会等待获取锁, 会立刻返回. 返回的结果集中, 去除被锁定的行  

注意: 这两个选项对于应用来说并不安全. 一个会返回异常, 另一个返回的结果集和数据库的状态并不一致     
使用示例:  
```sql
# 事务 1:
mysql> CREATE TABLE t (i INT, PRIMARY KEY (i)) ENGINE = InnoDB;
mysql> INSERT INTO t (i) VALUES(1),(2),(3);
mysql> START TRANSACTION;
mysql> SELECT * FROM t WHERE i = 2 FOR UPDATE;
+---+
| i |
+---+
| 2 |
+---+

# 事务 2:

mysql> START TRANSACTION;
mysql> SELECT * FROM t WHERE i = 2 FOR UPDATE NOWAIT;
ERROR 3572 (HY000): Do not wait for lock.

# 事务 3:
mysql> START TRANSACTION;
mysql> SELECT * FROM t FOR UPDATE SKIP LOCKED;
+---+
| i |
+---+
| 1 |
| 3 |
+---+
```

